In [1]:
# -*- coding: utf-8 -*-
import os
import yaml
import collections

import numpy as np
import pandas as pd

from datetime import date, timedelta, datetime
import glob
import json 

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import plotly.graph_objs as go
import dash_table

# -*- coding: utf-8 -*-
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

ENV_FILE = '../env.yaml'
with open(ENV_FILE) as f:
    params = yaml.load(f, Loader=yaml.FullLoader)

# Initialisation des chemins vers les fichiers
ROOT_DIR = os.path.dirname(os.path.abspath(ENV_FILE))
DATA_FILE = os.path.join(ROOT_DIR,
                         params['directories']['question_answering'],
                         params['files']['cord19_answers'])

# Lecture du fichier de données
with open(DATA_FILE, mode='r') as file:
     cord19_answers_json = json.loads(file.read())

tasks_questions = {
    tq['task']: [q['question'] for q in tq['questions']] for tq in cord19_answers_json['data']
}

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)


app.layout = html.Div([
    html.Label("Select another task"),
    dcc.Dropdown(
        id='tasks_dropdown',
        options=[{'label': k, 'value': k} for k in tasks_questions.keys()],
        value='What do we know about non-pharmaceutical interventions?'
    ),

    html.Hr(),
    html.Label("Select another question"),
    dcc.Dropdown(id='questions_dropdown'),

    html.Hr(),

    html.Div(id='display-selected-values')
])

@app.callback(
    Output('questions_dropdown', 'options'),
    [Input('tasks_dropdown', 'value')])
def set_cities_options(selected_task):
    return [{'label': i, 'value': i} for i in tasks_questions[selected_task]]


@app.callback(
    Output('questions_dropdown', 'value'),
    [Input('questions_dropdown', 'options')])
def set_cities_value(available_options):
    return available_options[0]['value']


@app.callback(
    Output('display-selected-values', 'children'),
    [Input('tasks_dropdown', 'value'),
     Input('questions_dropdown', 'value')])
def set_display_children(selected_task, selected_question):
    return u'{} is a city in {}'.format(
        selected_question, selected_task,
    )


if __name__ == '__main__':
    app.run_server(debug=True)
